In [37]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By  
import time
import urllib
import warnings

In [38]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [15]:
def download_slp(url_loc, dl = True):
    with urllib.request.urlopen(url_loc) as url:
        name = url_loc[url_loc.find('_') + 1:]
        if dl == True:
            data = open("Data/" + name, 'wb')
            data.write(url.read())
            data.close()
        else:
            print(name)

In [14]:
def page_download(soup):
    empty = True
    for i, link in enumerate(soup.findAll('a')):
        file_url = link.get('href')
        if not (file_url is None) and (file_url[:30] == 'https://storage.googleapis.com'):
            download_slp(file_url.replace(' ','%20'), False)
            if empty:
                empty = False
    return empty

In [51]:
def scrape_tournament(driver):
    counter = 1
    empty = False
    while not empty:
        html = driver.page_source
        soup = bs(html)
        empty = page_download(soup)
        if not empty:
            ui_buttons = driver.find_elements_by_class_name('MuiSvgIcon-root')
            ui_buttons[-1].click()
            time.sleep(5)
            counter += 1
        else:
            print(f'EMPTY PAGE FOUND ON {counter}')
    return counter

In [52]:
driver = webdriver.Firefox()
url = 'https://slippi.gg/' 
driver.get(url)
#
button_ui = driver.find_element_by_xpath('/html/body/div[1]/div/header/div/header/div/button')
button_ui.click()
#tournament button
button_tournament = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/ul/div[2]')
button_tournament.click()

time.sleep(5)

main_empty = False

while not main_empty:
    tournament_buttons = driver.find_elements_by_partial_link_text('Browse all games')
    tournament_count = len(tournament_buttons)
    
    if tournament_count != 0:
        for t in range(tournament_count):
            #enter page
            tournament_buttons = driver.find_elements_by_partial_link_text('Browse all games')
            tournament = tournament_buttons[t]
            tournament.click()
            time.sleep(5)
            back_counter = scrape_tournament(driver)
            #return to main
            for _ in range(back_counter):
                driver.back()
                time.sleep(1)

        print('NEXT TOURNAMENT PAGE')
        try:
            ui_buttons = driver.find_elements_by_class_name('MuiSvgIcon-root')
            ui_buttons[-1].click()
        except:
            main_empty = True
        
print('Done')

NEXT TOURNAMENT PAGE
NEXT TOURNAMENT PAGE
NEXT TOURNAMENT PAGE


ElementClickInterceptedException: Message: Element <svg class="MuiSvgIcon-root MuiPaginationItem-icon"> is not clickable at point (462,575) because another element <li> obscures it
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.jsm:282:5
webdriverClickElement@chrome://remote/content/marionette/interaction.js:166:11
interaction.clickElement@chrome://remote/content/marionette/interaction.js:125:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:203:24
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:91:31
